In [2]:
from iquhack_scoring import MoveScorer
from bloqade import move
import math
from kirin.passes import aggressive
import matplotlib as plt

In [3]:
pi = math.pi

In [10]:
@move.vmove()
def circ1():
    q = move.NewQubitRegister(3)
    
    # Optimized qubit indices in storage (Nstorage = 50)
    indices = [0, 1, 2]  # Spread evenly to minimize movement
    qubits = [q[0], q[1], q[2]]
    
    # Initialize qubits at new indices
    state = move.Init(qubits=qubits, indices=indices)
    
    # Move qubits to optimal gate zone positions (Ngates = 20)
    # gate_indices = [0, 10, 19]  # Spaced optimally for gate interactions
    state.gate[[0,1,2]] = move.Move(state.storage[[0,1,2]])
    
    # Apply first GlobalCZ operation
    state = move.core.GlobalCZ(state)
    
    # Apply LocalXY operations to qubit 1 at gate index 10
    state = move.LocalXY(atom_state=state, x_exponent=0.5 * pi, axis_phase_exponent=0.5 * pi, indices=[1])
    state = move.LocalXY(atom_state=state, x_exponent=1.0 * pi, axis_phase_exponent=0.0, indices=[1])
    
    # Move qubit 0 back to storage from gate index 0
    state.storage[[0]] = move.Move(state.gate[[0]])
    
    # Apply second GlobalCZ operation
    state = move.GlobalCZ(state)
    
    # Apply final LocalXY operations to qubit 1 at gate index 10
    state = move.LocalXY(atom_state=state, x_exponent=0.5 * pi, axis_phase_exponent=0.5 * pi, indices=[1])
    state = move.LocalXY(atom_state=state, x_exponent=1.0 * pi, axis_phase_exponent=0.0, indices=[1])
    
    # Execute the optimized circuit
    move.Execute(state)


In [11]:
expected_qasm = '''
OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [q(0), q(1), q(2)]
qreg q[3];


cz q[0],q[1];
cx q[2],q[1];
'''

In [12]:
analysis = MoveScorer(circ1)
analysis.score()
# ani = analysis.animate()
# ani.save("log_depth_GHZ.mp4")
# plt.show()


OPENQASM 2.0

;

include

"qelib1.inc"

;

...

1

]

;

// Local XY

{'time': 6.495686610870277,
 'ntouches': 4,
 'nmoves': 2,
 'apply_cz': 1,
 'apply_local_xy': 4,
 'overall': 7.499137322174056}